**Troubleshooting Scripts - Disk Performance**

Dmitri V. Korotkevitch (MCM, MVP)

email: [dk@aboutsqlserver.com](mailto:dk@aboutsqlserver.com)      blog: [https://aboutsqlserver.com](https://aboutsqlserver.com/) code: [https://github.com/aboutsqlserver/code](https://github.com/aboutsqlserver/code)

SQL Server Advanced Troubleshooting and Performance Tuning (O'Reilly, 2022)      ISBN: 978-1098101923

**I/O statistics since the time of last SQL Server restart** 

This may be useful in some cases; however, keep in mind that the numbers will be averaged across large time interval.

In [ ]:
SELECT 
	fs.database_id AS [DB ID]
	,fs.file_id AS [File Id]
	,mf.name AS [File Name]
	,mf.physical_name AS [File Path]
	,mf.type_desc AS [Type]
	,fs.sample_ms AS [Time]
	,fs.num_of_reads AS [Reads]
	,fs.num_of_bytes_read AS [Read Bytes]
	,fs.num_of_writes AS [Writes]
	,fs.num_of_bytes_written AS [Written Bytes]
	,fs.num_of_reads + fs.num_of_writes AS [IO Count]
	,CONVERT(DECIMAL(5,2),100.0 * fs.num_of_bytes_read / 
		(fs.num_of_bytes_read + fs.num_of_bytes_written)) AS [Read %]
	,CONVERT(DECIMAL(5,2),100.0 * fs.num_of_bytes_written / 
		(fs.num_of_bytes_read + fs.num_of_bytes_written)) AS [Write %]
	,fs.io_stall_read_ms AS [Read Stall]
	,fs.io_stall_write_ms AS [Write Stall]
	,CASE WHEN fs.num_of_reads = 0 
		THEN 0.000
		ELSE CONVERT(DECIMAL(12,3),1.0 * 
			fs.io_stall_read_ms / fs.num_of_reads) 
	END AS [Avg Read Stall] 
	,CASE WHEN fs.num_of_writes = 0 
		THEN 0.000
		ELSE CONVERT(DECIMAL(12,3),1.0 * 
			fs.io_stall_write_ms / fs.num_of_writes) 
	END AS [Avg Write Stall] 
FROM 
	sys.dm_io_virtual_file_stats(null,null) fs JOIN 
		sys.master_files mf WITH (NOLOCK) ON
			fs.database_id = mf.database_id AND
			fs.file_id = mf.file_id     
	JOIN sys.databases d WITH (NOLOCK) ON
		d.database_id = fs.database_id  
WHERE
	fs.num_of_reads + fs.num_of_writes > 0
OPTION (MAXDOP 1, RECOMPILE);

Getting snapshot of I/O subsystem statistics

Change time in WAITFOR DELAY statement as needed. I usually start with 1-5 minutes in balanced workload.

_Stall_ columns indicate latency of I/O requests. Lower is better

In [ ]:
IF OBJECT_ID(N'tempdb..#Snapshot') IS NOT NULL
	DROP TABLE #Snapshot;
GO

CREATE TABLE #Snapshot
(
	database_id SMALLINT NOT NULL,
	file_id SMALLINT NOT NULL,
	num_of_reads BIGINT NOT NULL,
	num_of_bytes_read BIGINT NOT NULL,
	io_stall_read_ms BIGINT NOT NULL,
	num_of_writes BIGINT NOT NULL,
	num_of_bytes_written BIGINT NOT NULL,
	io_stall_write_ms BIGINT NOT NULL
);

INSERT INTO #Snapshot(database_id,file_id,num_of_reads,num_of_bytes_read
	,io_stall_read_ms,num_of_writes,num_of_bytes_written,io_stall_write_ms)
	SELECT database_id,file_id,num_of_reads,num_of_bytes_read
		,io_stall_read_ms,num_of_writes,num_of_bytes_written,io_stall_write_ms
	FROM sys.dm_io_virtual_file_stats(NULL,NULL)
OPTION (MAXDOP 1, RECOMPILE);

-- Set test interval (1 minute). Use larger intervals as needed
WAITFOR DELAY '00:01:00.000';

;WITH Stats(db_id, file_id, Reads, ReadBytes, Writes
	,WrittenBytes, ReadStall, WriteStall)
as
(
	SELECT
		s.database_id, s.file_id
		,fs.num_of_reads - s.num_of_reads
		,fs.num_of_bytes_read - s.num_of_bytes_read
		,fs.num_of_writes - s.num_of_writes
		,fs.num_of_bytes_written - s.num_of_bytes_written
		,fs.io_stall_read_ms - s.io_stall_read_ms
		,fs.io_stall_write_ms - s.io_stall_write_ms
	FROM
		#Snapshot s JOIN  sys.dm_io_virtual_file_stats(NULL, NULL) fs ON
			s.database_id = fs.database_id and s.file_id = fs.file_id
)
SELECT
	s.db_id AS [DB ID], d.name AS [Database]
	,mf.name AS [File Name], mf.physical_name AS [File Path]
	,mf.type_desc AS [Type], s.Reads 
	,CONVERT(DECIMAL(12,3), s.ReadBytes / 1048576.) AS [Read MB]
	,CONVERT(DECIMAL(12,3), s.WrittenBytes / 1048576.) AS [Written MB]
	,s.Writes, s.Reads + s.Writes AS [IO Count]
	,CONVERT(DECIMAL(5,2),100.0 * s.ReadBytes / 
			(s.ReadBytes + s.WrittenBytes)) AS [Read %]
	,CONVERT(DECIMAL(5,2),100.0 * s.WrittenBytes / 
			(s.ReadBytes + s.WrittenBytes)) AS [Write %]
	,s.ReadStall AS [Read Stall]
	,s.WriteStall AS [Write Stall]
	,CASE WHEN s.Reads = 0 
		THEN 0.000
		ELSE CONVERT(DECIMAL(12,3),1.0 * s.ReadStall / s.Reads) 
	END AS [Avg Read Stall] 
	,CASE WHEN s.Writes = 0 
		THEN 0.000
		ELSE CONVERT(DECIMAL(12,3),1.0 * s.WriteStall / s.Writes) 
	END AS [Avg Write Stall] 
FROM
	Stats s JOIN  sys.master_files mf WITH (NOLOCK) ON
		s.db_id = mf.database_id and
		s.file_id = mf.file_id
	JOIN  sys.databases d WITH (NOLOCK) ON 
		s.db_id = d.database_id  
WHERE -- Only display files with more than 20MB throughput. Increase with larger sample times
	(s.ReadBytes + s.WrittenBytes) > 20 * 1048576
ORDER BY
	s.db_id, s.file_id
OPTION (MAXDOP 1, RECOMPILE);


I/O related performace counters

In [ ]:
IF OBJECT_ID(N'tempdb..#PerfCntrs') IS NOT NULL
	DROP TABLE #PerfCntrs;
GO

CREATE TABLE #PerfCntrs
(
	collected_time DATETIME2(7) NOT NULL DEFAULT SYSDATETIME(),
	object_name SYSNAME NOT NULL,
	counter_name SYSNAME NOT NULL,
	instance_name SYSNAME NOT NULL,
	cntr_value BIGINT NOT NULL,
	PRIMARY KEY (object_name, counter_name, instance_name)
);

;WITH Counters(obj_name, ctr_name)
AS
(
	SELECT C.obj_name, C.ctr_name
	FROM 
	(
		VALUES
			('SQLServer:Buffer Manager','Checkpoint pages/sec')
			,('SQLServer:Buffer Manager','Background writer pages/sec')
			,('SQLServer:Buffer Manager','Lazy writes/sec')
			,('SQLServer:Buffer Manager','Page reads/sec')
			,('SQLServer:Buffer Manager','Page writes/sec')
			,('SQLServer:Buffer Manager','Readahead pages/sec')
			,('SQLServer:Databases','Log Flushes/sec') -- For all DBs
			,('SQLServer:Databases','Log Bytes Flushed/sec') -- For all DBs
			,('SQLServer:Databases','Log Flush Write Time (ms)') -- For all DBs
			,('SQLServer:Databases','Transactions/sec') -- For all DBs
			,('SQLServer:SQL Statistics','Batch Requests/sec') 
	) C(obj_name, ctr_name)
)
INSERT INTO #PerfCntrs(object_name,counter_name,instance_name,cntr_value)
	SELECT 
		pc.object_name, pc.counter_name, pc.instance_name, pc.cntr_value
	FROM 
		sys.dm_os_performance_counters pc WITH (NOLOCK) JOIN Counters c ON
			pc.counter_name = c.ctr_name AND pc.object_name = c.obj_name;

WAITFOR DELAY '00:00:01.000';

;WITH Counters(obj_name, ctr_name)
AS
(
	SELECT C.obj_name, C.ctr_name
	FROM 
	(
		VALUES
			('SQLServer:Buffer Manager','Checkpoint pages/sec')
			,('SQLServer:Buffer Manager','Background writer pages/sec')
			,('SQLServer:Buffer Manager','Lazy writes/sec')
			,('SQLServer:Buffer Manager','Page reads/sec')
			,('SQLServer:Buffer Manager','Page writes/sec')
			,('SQLServer:Buffer Manager','Readahead pages/sec')
			,('SQLServer:Databases','Log Flushes/sec') -- For all DBs
			,('SQLServer:Databases','Log Bytes Flushed/sec') -- For all DBs
			,('SQLServer:Databases','Log Flush Write Time (ms)') -- For all DBs
			,('SQLServer:Databases','Transactions/sec') -- For all DBs
			,('SQLServer:SQL Statistics','Batch Requests/sec') 
	) C(obj_name, ctr_name)
)
SELECT 
	pc.object_name, pc.counter_name, pc.instance_name
	,CASE pc.cntr_type
		WHEN 272696576 THEN 
			(pc.cntr_value - h.cntr_value) * 1000 / 
				DATEDIFF(MILLISECOND,h.collected_time,SYSDATETIME())
		WHEN 65792 THEN 
			pc.cntr_value
		ELSE NULL
	END as cntr_value
FROM 
	sys.dm_os_performance_counters pc WITH (NOLOCK) JOIN Counters c ON
		pc.counter_name = c.ctr_name AND pc.object_name = c.obj_name
	JOIN #PerfCntrs h ON
		pc.object_name = h.object_name AND
		pc.counter_name = h.counter_name AND
		pc.instance_name = h.instance_name
ORDER BY
	pc.object_name, pc.counter_name, pc.instance_name
OPTION (RECOMPILE);

**List of I/O requests**

io\_pending column indicates if OS I/O API call has been completed and request is waiting for available scheduler to finalize. 

Undocumented io\_pending\_ms\_ticks indicates the duration

In [ ]:
SELECT    
    ip.io_type
    ,ip.io_pending
    ,io_pending_ms_ticks
    ,ip.scheduler_address
    ,ip.io_handle
    ,s.scheduler_id
    ,s.cpu_id
    ,s.pending_disk_io_count
    ,er.session_id
    ,er.command
    ,er.cpu_time
    ,SUBSTRING(
        qt.text, 
        (er.statement_start_offset / 2) + 1,
            ((CASE er.statement_end_offset
                WHEN -1 THEN DATALENGTH(qt.text)
                ELSE er.statement_end_offset
            END - er.statement_start_offset) / 2) + 1
    ) AS [statement]
FROM 
    sys.dm_io_pending_io_requests ip WITH (NOLOCK)
        LEFT JOIN sys.dm_os_schedulers s WITH (NOLOCK) ON 
            ip.scheduler_address = s.scheduler_address
        LEFT JOIN sys.dm_exec_requests er ON 
            s.scheduler_id = er.scheduler_id
        OUTER APPLY 
            sys.dm_exec_sql_text(er.sql_handle) qt
OPTION (MAXDOP 1, RECOMPILE);

**Page Life Expectancy (PLE)**

In [ ]:
SELECT object_name, counter_name, instance_name, cntr_value as [PLE(sec)]
FROM sys.dm_os_performance_counters WITH (NOLOCK) 
WHERE counter_name = 'Page life expectancy'
OPTION (MAXDOP 1, RECOMPILE);

**Suspect Pages**

In [ ]:
SELECT 
    DB_NAME(database_id) AS [database]
    ,file_id
    ,page_id
    ,event_type
    ,error_count
    , last_update_date 
FROM msdb.dbo.suspect_pages WITH (NOLOCK)
ORDER BY database_id 
OPTION (MAXDOP 1,RECOMPILE);